### Init

In [ ]:
import kutils
from kutils import model_helper as mh
from kutils import applications as apps
from kutils import tensor_ops as ops
from kutils import generic as gen

In [ ]:
import pandas as pd, numpy as np, os
root_path = '/mnt/home/research/ava-mlsp/'
dataset = root_path + 'metadata/AVA_data_official_test.csv';
images_path = root_path + 'images/'
ids = pd.read_csv(dataset)

### Load combined model

In [ ]:
# load base model
model_name = 'mlsp_wide_orig'
input_shape = (None, None, 3)
model_base = apps.model_inceptionresnet_pooled(input_shape)
pre   = apps.process_input[apps.InceptionResNetV2]

# MODEL DEF
from keras.layers import Input, GlobalAveragePooling2D
from keras.models import Model

input_feats = Input(shape=(5,5,16928), dtype='float32')
x = apps.inception_block(input_feats, size=1024)
x = GlobalAveragePooling2D(name='final_GAP')(x)

pred = apps.fc_layers(x, name       = 'head',
                      fc_sizes      = [2048, 1024, 256,  1],
                      dropout_rates = [0.25, 0.25, 0.5, 0],
                      batch_norm    = 2)

model = Model(inputs  = input_feats, 
              outputs = pred)

gen_params = dict(batch_size    = 1,
                  data_path     = images_path,                  
                  process_fn    = pre,
                  input_shape   = input_shape,
                  inputs        = 'image_name',
                  outputs       = 'MOS', 
                  fixed_batches = False)

helper = mh.ModelHelper(model, model_name, ids, 
                        gen_params = gen_params)

# load head model
helper.load_model(model_name = root_path + \
                               'models/irnv2_mlsp_wide_orig/model')

# join base and head models
helper.model = Model(inputs  = model_base.input, 
                     outputs = model(model_base.output))

### Read weights

In [ ]:
helper.gen_params.verbose = True
_, _, SRCC_test, PLCC_test, ACC_test =\
    apps.test_rating_model(helper, accuracy_thresh = 5, 
                           ids = ids[ids.set == 'test'])